In [3]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import cv2
from models import ResNet18
from config.unlabel_config import ensemble_config as ecfg

In [5]:
valid_transforms = [
            transforms.Resize(64),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
val_transform = transforms.Compose(valid_transforms)
val_set = datasets.ImageFolder('/home/stu6/EuroSAT_PRML24/Task_B/val', val_transform)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=4)

In [11]:
def val_ensemble(models, weights, val_loader, criterion):
    for model in models:
        model.train(False)
    total_loss = 0.0
    total_correct = 0
    for inputs, labels in val_loader:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        for i, model in enumerate(models):
            outputs = weights[i] * model(inputs)
            _, prediction = torch.max(outputs, 1)
            predictions += prediction * weights[i]
        _, predictions = torch.max(predictions, 1)
        total_correct += torch.sum(predictions == labels.data)
        
    val_loss = total_loss / len(val_loader.dataset)
    val_acc = total_correct.double() / len(val_loader.dataset)
    return val_loss, val_acc.item()

In [13]:
paths = ecfg['paths']
models = []
for path in paths:
    model = torch.load(path)
    model = model.to('cuda')
    models.append(model)
    
val_loss, val_acc = val_ensemble(models, 
                                 weights=torch.tensor([1,1,1]), 
                                 val_loader=val_loader, 
                                 criterion=nn.CrossEntropyLoss())

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [12]:
print(val_loss)
print(val_acc)

1.1434409968058268
0.7117283950617284
